In [7]:
import os
import random
import numpy as np

import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

from net import weights_init, Generator, Discriminator

In [8]:
def onehot_encode(label, device, n_class=10):
    """
    カテゴリカル変数のラベルをOne-Hoe形式に変換する
    :param label: 変換対象のラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :param n_class: ラベルのクラス数
    :return:
    """
    eye = torch.eye(n_class, device=device)
    # ランダムベクトルあるいは画像と連結するために(B, c_class, 1, 1)のテンソルにして戻す
    return eye[label].view(-1, n_class, 1, 1)   

In [9]:
def concat_image_label(image, label, device, n_class=10):
    """
    画像とラベルを連結する
    :param image:　画像
    :param label: ラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :param n_class: ラベルのクラス数
    :return:　画像とラベルをチャネル方向に連結したテンソル
    """
    B, C, H, W = image.shape    # 画像テンソルの大きさを取得
    
    oh_label = onehot_encode(label, device)         # ラベルをOne-Hotベクトル化
    oh_label = oh_label.expand(B, n_class, H, W)    # 画像のサイズに合わせるようラベルを拡張する
    return torch.cat((image, oh_label), dim=1)      # 画像とラベルをチャネル方向（dim=1）で連結する

In [10]:
def concat_noise_label(noise, label, device):
    """
    ノイズ（ランダムベクトル）とラベルを連結する
    :param noise: ノイズ
    :param label: ラベル
    :param device: 学習に使用するデバイス。CPUあるいはGPU
    :return:　ノイズとラベルを連結したテンソル
    """
    oh_label = onehot_encode(label, device)     # ラベルをOne-Hotベクトル化
    return torch.cat((noise, oh_label), dim=1)  # ノイズとラベルをチャネル方向（dim=1）で連結する

In [11]:
workers = 2
batch_size = 50
nz = 100
nch_g = 64
nch_d = 64
n_epoch = 200
lr = 0.0002
beta1 = 0.5
outf = './result_cgan'

try:
    os.makedirs(outf)
except OSError:
    pass

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [12]:
trainset = dset.STL10(root='../../dataset/stl10_root', download=True, split='train',
                      transform=transforms.Compose([
                          transforms.RandomResizedCrop(64, scale=(88/96, 1.0), ratio=(1., 1.)),
                          transforms.RandomHorizontalFlip(),
                          transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                          transforms.ToTensor(),
                          transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                      ]))   # ラベルを使用するのでunlabeledを含めない
testset = dset.STL10(root='../../dataset/stl10_root', download=True, split='test',
                     transform=transforms.Compose([
                         transforms.RandomResizedCrop(64, scale=(88/96, 1.0), ratio=(1., 1.)),
                         transforms.RandomHorizontalFlip(),
                         transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
                         transforms.ToTensor(),
                         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                     ]))
dataset = trainset + testset

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=int(workers))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Files already downloaded and verified
Files already downloaded and verified
device: cuda:0


In [13]:
# 生成器G。ランダムベクトルとラベルを連結したベクトルから贋作画像を生成する
netG = Generator(nz=nz+10, nch_g=nch_g).to(device)   # 入力ベクトルの次元は、ランダムベクトルの次元nzにクラス数10を加算したもの
netG.apply(weights_init)
print(netG)

Generator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): ConvTranspose2d(110, 512, kernel_size=(4, 4), stride=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer1): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer2): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer3): Sequential(
      (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer4): Sequential(
      (0): ConvTranspose2d(64, 3, ker

In [14]:
# 識別器D。画像とラベルを連結したテンソルが、元画像か贋作画像かを識別する
netD = Discriminator(nch=3+10, nch_d=nch_d).to(device)   # 入力テンソルのチャネル数は、画像のチャネル数3にクラス数10を加算したもの
netD.apply(weights_init)
print(netD)

Discriminator(
  (layers): ModuleDict(
    (layer0): Sequential(
      (0): Conv2d(13, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
    (layer1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (layer4): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
  )
)


In [15]:
criterion = nn.MSELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999), weight_decay=1e-5)

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)

fixed_label = [i for i in range(10)] * (batch_size // 10)  # 確認用のラベル。0〜9のラベルの繰り返し
fixed_label = torch.tensor(fixed_label, dtype=torch.long, device=device)

fixed_noise_label = concat_noise_label(fixed_noise, fixed_label, device)  # 確認用のノイズとラベルを連結

In [16]:
# 学習のループ
for epoch in range(n_epoch):
    for itr, data in enumerate(dataloader):
        real_image = data[0].to(device)     # 元画像
        real_label = data[1].to(device)     # 元画像に対応するラベル
        real_image_label = concat_image_label(real_image, real_label, device)   # 元画像とラベルを連結

        sample_size = real_image.size(0)
        noise = torch.randn(sample_size, nz, 1, 1, device=device)
        fake_label = torch.randint(10, (sample_size,), dtype=torch.long, device=device)     # 贋作画像生成用のラベル
        fake_noise_label = concat_noise_label(noise, fake_label, device)    # ノイズとラベルを連結
        
        real_target = torch.full((sample_size,), 1., device=device)
        fake_target = torch.full((sample_size,), 0., device=device)

        ############################
        # 識別器Dの更新
        ###########################
        netD.zero_grad()

        output = netD(real_image_label)     # 識別器Dで元画像とラベルの組み合わせに対する識別信号を出力
        errD_real = criterion(output, real_target)
        errD_real.backward()
        D_x = output.mean().item()

        fake_image = netG(fake_noise_label)     # 生成器Gでラベルに対応した贋作画像を生成
        fake_image_label = concat_image_label(fake_image, fake_label, device)   # 贋作画像とラベルを連結

        output = netD(fake_image_label.detach())    # 識別器Dで贋作画像とラベルの組み合わせに対する識別信号を出力
        errD_fake = criterion(output, fake_target)
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # 生成器Gの更新
        ###########################
        netG.zero_grad()
        
        output = netD(fake_image_label)     # 更新した識別器Dで改めて贋作画像とラベルの組み合わせに対する識別信号を出力
        errG = criterion(output, real_target)
        errG.backward()
        D_G_z2 = output.mean().item()
        
        optimizerG.step()

        print('[{}/{}][{}/{}] Loss_D: {:.3f} Loss_G: {:.3f} D(x): {:.3f} D(G(z)): {:.3f}/{:.3f}'
              .format(epoch + 1, n_epoch,
                      itr + 1, len(dataloader),
                      errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        if epoch == 0 and itr == 0:
            vutils.save_image(real_image, '{}/real_samples.png'.format(outf),
                              normalize=True, nrow=10)

    ############################
    # 確認用画像の生成
    ############################
    fake_image = netG(fixed_noise_label)    # 1エポック終了ごとに、指定したラベルに対応する贋作画像を生成する
    vutils.save_image(fake_image.detach(), '{}/fake_samples_epoch_{:03d}.png'.format(outf, epoch + 1),
                      normalize=True, nrow=10)

    ############################
    # モデルの保存
    ############################
    if (epoch + 1) % 50 == 0:
        torch.save(netG.state_dict(), '{}/netG_epoch_{}.pth'.format(outf, epoch + 1))
        torch.save(netD.state_dict(), '{}/netD_epoch_{}.pth'.format(outf, epoch + 1))

[1/200][1/260] Loss_D: 1.958 Loss_G: 31.280 D(x): 0.480 D(G(z)): 0.105/1.759
[1/200][2/260] Loss_D: 38.808 Loss_G: 4.630 D(x): 2.279 D(G(z)): 1.708/-0.496
[1/200][3/260] Loss_D: 4.445 Loss_G: 6.116 D(x): 0.578 D(G(z)): 0.088/-0.538
[1/200][4/260] Loss_D: 5.810 Loss_G: 1.817 D(x): 0.119 D(G(z)): 0.122/0.301
[1/200][5/260] Loss_D: 2.702 Loss_G: 2.153 D(x): 0.746 D(G(z)): 0.749/-0.288
[1/200][6/260] Loss_D: 2.315 Loss_G: 1.645 D(x): 0.224 D(G(z)): 0.421/-0.102
[1/200][7/260] Loss_D: 2.083 Loss_G: 1.403 D(x): 0.312 D(G(z)): 0.702/0.006
[1/200][8/260] Loss_D: 2.650 Loss_G: 3.322 D(x): 0.591 D(G(z)): 0.769/-0.604
[1/200][9/260] Loss_D: 1.830 Loss_G: 2.841 D(x): 0.399 D(G(z)): 0.586/-0.602
[1/200][10/260] Loss_D: 1.725 Loss_G: 1.237 D(x): 0.256 D(G(z)): 0.163/0.184
[1/200][11/260] Loss_D: 2.310 Loss_G: 6.015 D(x): 1.244 D(G(z)): 0.896/-1.237
[1/200][12/260] Loss_D: 2.983 Loss_G: 0.754 D(x): -0.156 D(G(z)): -0.334/0.616
[1/200][13/260] Loss_D: 2.825 Loss_G: 4.608 D(x): 1.149 D(G(z)): 1.261/-0.

[1/200][107/260] Loss_D: 0.570 Loss_G: 1.465 D(x): 1.157 D(G(z)): 0.550/-0.191
[1/200][108/260] Loss_D: 0.208 Loss_G: 1.415 D(x): 0.823 D(G(z)): -0.019/-0.174
[1/200][109/260] Loss_D: 0.229 Loss_G: 1.411 D(x): 1.070 D(G(z)): 0.061/-0.163
[1/200][110/260] Loss_D: 0.326 Loss_G: 0.668 D(x): 0.739 D(G(z)): -0.102/0.221
[1/200][111/260] Loss_D: 0.475 Loss_G: 1.993 D(x): 1.303 D(G(z)): 0.357/-0.400
[1/200][112/260] Loss_D: 0.472 Loss_G: 0.848 D(x): 0.527 D(G(z)): -0.221/0.182
[1/200][113/260] Loss_D: 0.485 Loss_G: 1.753 D(x): 1.102 D(G(z)): 0.339/-0.266
[1/200][114/260] Loss_D: 0.347 Loss_G: 1.054 D(x): 0.703 D(G(z)): -0.030/0.001
[1/200][115/260] Loss_D: 0.404 Loss_G: 1.656 D(x): 0.963 D(G(z)): 0.328/-0.243
[1/200][116/260] Loss_D: 0.444 Loss_G: 0.884 D(x): 0.883 D(G(z)): -0.134/0.094
[1/200][117/260] Loss_D: 0.397 Loss_G: 1.726 D(x): 1.333 D(G(z)): 0.166/-0.295
[1/200][118/260] Loss_D: 0.387 Loss_G: 0.575 D(x): 0.656 D(G(z)): -0.168/0.266
[1/200][119/260] Loss_D: 0.755 Loss_G: 3.495 D(x): 

[1/200][215/260] Loss_D: 0.125 Loss_G: 0.974 D(x): 0.962 D(G(z)): -0.061/0.019
[1/200][216/260] Loss_D: 0.187 Loss_G: 1.194 D(x): 0.862 D(G(z)): 0.217/-0.060
[1/200][217/260] Loss_D: 0.324 Loss_G: 0.802 D(x): 0.710 D(G(z)): -0.017/0.202
[1/200][218/260] Loss_D: 0.471 Loss_G: 2.396 D(x): 1.180 D(G(z)): 0.423/-0.527
[1/200][219/260] Loss_D: 0.369 Loss_G: 0.701 D(x): 0.803 D(G(z)): -0.413/0.177
[1/200][220/260] Loss_D: 0.153 Loss_G: 0.696 D(x): 0.941 D(G(z)): 0.203/0.176
[1/200][221/260] Loss_D: 0.273 Loss_G: 1.939 D(x): 1.269 D(G(z)): 0.263/-0.386
[1/200][222/260] Loss_D: 0.220 Loss_G: 1.090 D(x): 0.917 D(G(z)): -0.253/-0.029
[1/200][223/260] Loss_D: 0.190 Loss_G: 0.606 D(x): 0.778 D(G(z)): 0.032/0.255
[1/200][224/260] Loss_D: 0.232 Loss_G: 0.983 D(x): 0.913 D(G(z)): 0.280/0.020
[1/200][225/260] Loss_D: 0.196 Loss_G: 1.040 D(x): 0.744 D(G(z)): 0.161/-0.012
[1/200][226/260] Loss_D: 0.168 Loss_G: 1.350 D(x): 1.001 D(G(z)): 0.113/-0.146
[1/200][227/260] Loss_D: 0.340 Loss_G: 0.532 D(x): 0.5

[2/200][61/260] Loss_D: 0.372 Loss_G: 1.508 D(x): 1.056 D(G(z)): 0.382/-0.186
[2/200][62/260] Loss_D: 0.462 Loss_G: 1.146 D(x): 0.702 D(G(z)): 0.231/-0.031
[2/200][63/260] Loss_D: 0.708 Loss_G: 0.626 D(x): 0.446 D(G(z)): 0.369/0.246
[2/200][64/260] Loss_D: 0.690 Loss_G: 0.716 D(x): 0.547 D(G(z)): 0.392/0.235
[2/200][65/260] Loss_D: 0.396 Loss_G: 0.858 D(x): 0.784 D(G(z)): 0.156/0.117
[2/200][66/260] Loss_D: 0.665 Loss_G: 1.212 D(x): 0.874 D(G(z)): 0.578/-0.052
[2/200][67/260] Loss_D: 0.530 Loss_G: 0.996 D(x): 0.572 D(G(z)): 0.163/0.029
[2/200][68/260] Loss_D: 0.428 Loss_G: 0.772 D(x): 0.641 D(G(z)): 0.385/0.157
[2/200][69/260] Loss_D: 0.660 Loss_G: 0.688 D(x): 0.436 D(G(z)): 0.299/0.257
[2/200][70/260] Loss_D: 0.852 Loss_G: 1.022 D(x): 0.693 D(G(z)): 0.661/0.032
[2/200][71/260] Loss_D: 0.442 Loss_G: 0.934 D(x): 0.613 D(G(z)): 0.275/0.054
[2/200][72/260] Loss_D: 0.470 Loss_G: 0.656 D(x): 0.476 D(G(z)): 0.236/0.242
[2/200][73/260] Loss_D: 0.474 Loss_G: 1.069 D(x): 0.845 D(G(z)): 0.521/0.

[2/200][169/260] Loss_D: 0.531 Loss_G: 0.631 D(x): 0.981 D(G(z)): 0.582/0.226
[2/200][170/260] Loss_D: 0.259 Loss_G: 1.925 D(x): 1.192 D(G(z)): 0.317/-0.371
[2/200][171/260] Loss_D: 0.780 Loss_G: 0.381 D(x): 0.326 D(G(z)): -0.343/0.425
[2/200][172/260] Loss_D: 0.484 Loss_G: 0.847 D(x): 1.136 D(G(z)): 0.493/0.124
[2/200][173/260] Loss_D: 0.220 Loss_G: 1.042 D(x): 0.806 D(G(z)): 0.196/-0.002
[2/200][174/260] Loss_D: 0.154 Loss_G: 0.939 D(x): 0.843 D(G(z)): 0.057/0.045
[2/200][175/260] Loss_D: 0.178 Loss_G: 0.678 D(x): 0.760 D(G(z)): 0.151/0.195
[2/200][176/260] Loss_D: 0.170 Loss_G: 0.922 D(x): 0.923 D(G(z)): 0.272/0.046
[2/200][177/260] Loss_D: 0.145 Loss_G: 0.816 D(x): 0.769 D(G(z)): 0.048/0.105
[2/200][178/260] Loss_D: 0.110 Loss_G: 0.765 D(x): 0.851 D(G(z)): 0.151/0.144
[2/200][179/260] Loss_D: 0.116 Loss_G: 1.226 D(x): 1.091 D(G(z)): 0.112/-0.098
[2/200][180/260] Loss_D: 0.111 Loss_G: 1.118 D(x): 0.911 D(G(z)): 0.109/-0.043
[2/200][181/260] Loss_D: 0.227 Loss_G: 0.743 D(x): 0.703 D(

...

[4/200][181/260] Loss_D: 0.445 Loss_G: 0.540 D(x): 0.629 D(G(z)): 0.441/0.290
[4/200][182/260] Loss_D: 0.437 Loss_G: 0.619 D(x): 0.597 D(G(z)): 0.387/0.264
[4/200][183/260] Loss_D: 0.445 Loss_G: 0.459 D(x): 0.535 D(G(z)): 0.283/0.351
[4/200][184/260] Loss_D: 0.463 Loss_G: 0.523 D(x): 0.697 D(G(z)): 0.489/0.300
[4/200][185/260] Loss_D: 0.426 Loss_G: 0.663 D(x): 0.675 D(G(z)): 0.448/0.219
[4/200][186/260] Loss_D: 0.575 Loss_G: 0.354 D(x): 0.392 D(G(z)): 0.327/0.425
[4/200][187/260] Loss_D: 0.592 Loss_G: 0.478 D(x): 0.624 D(G(z)): 0.603/0.322
[4/200][188/260] Loss_D: 0.452 Loss_G: 0.526 D(x): 0.552 D(G(z)): 0.369/0.291
[4/200][189/260] Loss_D: 0.608 Loss_G: 0.369 D(x): 0.445 D(G(z)): 0.446/0.446
[4/200][190/260] Loss_D: 0.355 Loss_G: 0.614 D(x): 0.712 D(G(z)): 0.386/0.234
[4/200][191/260] Loss_D: 0.474 Loss_G: 0.671 D(x): 0.633 D(G(z)): 0.392/0.241
[4/200][192/260] Loss_D: 0.406 Loss_G: 0.821 D(x): 0.721 D(G(z)): 0.327/0.140
[4/200][193/260] Loss_D: 0.592 Loss_G: 0.407 D(x): 0.441 D(G(z))

[5/200][27/260] Loss_D: 0.563 Loss_G: 1.012 D(x): 0.807 D(G(z)): 0.563/0.040
[5/200][28/260] Loss_D: 0.882 Loss_G: 0.543 D(x): 0.327 D(G(z)): 0.370/0.402
[5/200][29/260] Loss_D: 0.561 Loss_G: 0.589 D(x): 0.640 D(G(z)): 0.342/0.330
[5/200][30/260] Loss_D: 0.499 Loss_G: 0.602 D(x): 0.690 D(G(z)): 0.331/0.291
[5/200][31/260] Loss_D: 0.420 Loss_G: 0.894 D(x): 0.898 D(G(z)): 0.447/0.067
[5/200][32/260] Loss_D: 0.527 Loss_G: 0.493 D(x): 0.444 D(G(z)): 0.356/0.310
[5/200][33/260] Loss_D: 0.646 Loss_G: 0.193 D(x): 0.316 D(G(z)): 0.288/0.612
[5/200][34/260] Loss_D: 0.534 Loss_G: 0.409 D(x): 0.733 D(G(z)): 0.606/0.383
[5/200][35/260] Loss_D: 0.585 Loss_G: 0.703 D(x): 0.652 D(G(z)): 0.568/0.195
[5/200][36/260] Loss_D: 0.656 Loss_G: 0.453 D(x): 0.380 D(G(z)): 0.413/0.356
[5/200][37/260] Loss_D: 0.293 Loss_G: 0.560 D(x): 0.663 D(G(z)): 0.290/0.271
[5/200][38/260] Loss_D: 0.376 Loss_G: 0.667 D(x): 0.709 D(G(z)): 0.369/0.226
[5/200][39/260] Loss_D: 0.374 Loss_G: 0.748 D(x): 0.734 D(G(z)): 0.430/0.151

Process Process-10:
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/sika/anaconda3/envs/torch_env/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = col

[5/200][93/260] Loss_D: 0.584 Loss_G: 0.550 D(x): 0.729 D(G(z)): 0.633/0.271
[5/200][94/260] Loss_D: 0.691 Loss_G: 0.415 D(x): 0.410 D(G(z)): 0.467/0.380


KeyboardInterrupt: 